In [1]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import neurokit2 as nk
import pyedflib
import pandas as pd
import os

## Data

In [2]:
# directory of EDF files
#edf_directory = '../datasetsPart2/valu3s/vitaport/'
edf_directory = '../datasets_3/DROZY/psg/'

In [3]:
# path of .csv files
csv_directory = '../datasets_2/valu3s/vitaport/filtered_signals/'

In [3]:
# duration of each interval in seconds
interval_duration = 60 

# initialize an empty df to store all features
feature_dfs = []

# sampling Rate
#sampling_rate = 256
sampling_rate = 512

## Process Freq Domain ECG

In [4]:
# function extract featrures from ECG
def extract_hrv_features(signal, sampling_rate, interval_duration):
    # number of intrevals
    interval_samples = int(interval_duration * sampling_rate)
    features = [] 
    
    # loop to extract features for each intreval 
    for start in range(0, len(signal), interval_samples):
        end = start + interval_samples
        if end > len(signal):
            break
        
        interval_signal = signal[start:end]
        if len(interval_signal) < interval_samples:
            continue
        
        try:
            # extract features 
            signals, info = nk.ecg_process(interval_signal, sampling_rate=sampling_rate)
            hrv_features = nk.hrv_frequency(info['ECG_R_Peaks'], sampling_rate=sampling_rate, normalize=True)
            hrv_features['Interval_Start'] = start / sampling_rate
            hrv_features['Interval_End'] = end / sampling_rate

            # add extrated features to the DataFrame
            features.append(hrv_features)
        except Exception as e:
            print(f"Error processing interval {start}-{end}: {e}")
            continue

    return features

# loop through all .edf files in the directory
for filename in os.listdir(edf_directory):
    if filename.endswith('.edf'):
        file_path = os.path.join(edf_directory, filename)
        
        # load the .edf file
        f = pyedflib.EdfReader(file_path)
        
        # get signal labels and find ECG signal index
        signal_labels = f.getSignalLabels()
        if 'ECG' in signal_labels:
            ecg_signal_index = signal_labels.index('ECG')
            ecg_signal = f.readSignal(ecg_signal_index)
            sampling_rate = f.getSampleFrequency(ecg_signal_index)
        else:
            print(f"No ECG signal found in {filename}")
            continue
        
        # close the EDF file
        f.close()

        # duration of the ECG signal in seconds
        ecg_duration = len(ecg_signal) / sampling_rate
        print(f"ECG duration for {filename}: {ecg_duration} seconds")
        
        # extract HRV features for each interval
        interval_features = extract_hrv_features(ecg_signal, sampling_rate, interval_duration)

        # number of intervals
        num_intervals = len(interval_features)
        print(f"Number of intervals for {filename}: {num_intervals}")
        
        if interval_features:
            # convert list of feature dictionaries to DataFrame and add filename
            interval_features_df = pd.concat([pd.DataFrame(features) for features in interval_features])
            interval_features_df['Filename'] = filename
        
            # append the features DataFrame to the list
            feature_dfs.append(interval_features_df)
        else:
            print(f"No valid intervals found in {filename}")

ECG duration for 1-1.edf: 600.0 seconds
Number of intervals for 1-1.edf: 10
ECG duration for 1-2.edf: 600.0 seconds
Number of intervals for 1-2.edf: 10
ECG duration for 1-3.edf: 600.0 seconds
Number of intervals for 1-3.edf: 10
ECG duration for 10-1.edf: 600.0 seconds
Number of intervals for 10-1.edf: 10
ECG duration for 10-3.edf: 600.0 seconds
Number of intervals for 10-3.edf: 10
ECG duration for 11-1.edf: 600.0 seconds
Number of intervals for 11-1.edf: 10
ECG duration for 11-2.edf: 600.0 seconds
Number of intervals for 11-2.edf: 10
ECG duration for 11-3.edf: 600.0 seconds
Number of intervals for 11-3.edf: 10
ECG duration for 12-1.edf: 600.0 seconds
Number of intervals for 12-1.edf: 10
ECG duration for 13-1.edf: 600.0 seconds
Number of intervals for 13-1.edf: 10
ECG duration for 13-2.edf: 600.0 seconds
Number of intervals for 13-2.edf: 10
ECG duration for 14-1.edf: 600.0 seconds
Number of intervals for 14-1.edf: 10
ECG duration for 14-2.edf: 600.0 seconds
Number of intervals for 14-2.

## Save Features Dataset EDF

In [5]:
# concatenate all feature DataFrames into one DataFrame
all_features_df = pd.concat(feature_dfs, ignore_index=True)

# save the features DataFrame to a CSV file
all_features_df.to_csv('DROZY_hrv_freq_domain_1_min.csv', index=False)

print("Saved features on 'DROZY_hrv_freq_domain_1_min.csv'.")

Saved features on 'DROZY_hrv_freq_domain_1_min.csv'.


## Process Freq Domain CSV

In [5]:
# function extract featrures from ECG
def extract_hrv_features(signal, sampling_rate, interval_duration):
    # number of intrevals
    start_sample = int(30 * sampling_rate)
    interval_samples = int(interval_duration * sampling_rate)
    features = [] 
    
    # loop to extract features for each intreval 
    for start in range(start_sample, len(signal), interval_samples):
        end = start + interval_samples
        if end > len(signal):
            break
        
        interval_signal = signal[start:end]
        if len(interval_signal) < interval_samples:
            continue
        
        try:
            # extract features 
            signals, info = nk.ecg_process(interval_signal, sampling_rate=sampling_rate)
            hrv_features = nk.hrv_frequency(info['ECG_R_Peaks'], sampling_rate=sampling_rate, normalize=True)
            hrv_features['Interval_Start'] = start / sampling_rate
            hrv_features['Interval_End'] = end / sampling_rate

            # add extrated features to the DataFrame
            features.append(hrv_features)
        except Exception as e:
            print(f"Error processing interval {start}-{end}: {e}")
            continue

    return features

# loop all edf files 
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_directory, filename)
        
        # load the CSV file 
        try:
            # read CSV 
            df = pd.read_csv(file_path, header=None)
            ecg_signal = df[0].values 
        except Exception as e:
            print(f"Error loading {filename}: {e}")
            continue

        # duration of the ECG signal in seconds
        ecg_duration = len(ecg_signal) / sampling_rate
        print(f"ECG duration for {filename}: {ecg_duration} seconds")
        
        # extract HRV features for each interval
        interval_features = extract_hrv_features(ecg_signal, sampling_rate, interval_duration)

        # number of intervals
        num_intervals = len(interval_features)
        print(f"Number of intervals for {filename}: {num_intervals}")
        
        if interval_features:
            # convert list of feature dictionaries to DataFrame and add filename
            interval_features_df = pd.concat([pd.DataFrame(features) for features in interval_features])
            interval_features_df['Filename'] = filename
        
            # append the features DataFrame to the list
            feature_dfs.append(interval_features_df)
        else:
            print(f"No valid intervals found in {filename}")

ECG duration for fp01_1.csv: 3694.578125 seconds
Number of intervals for fp01_1.csv: 61
ECG duration for fp01_2.csv: 3576.703125 seconds
Number of intervals for fp01_2.csv: 59
ECG duration for fp01_3.csv: 3574.078125 seconds
Number of intervals for fp01_3.csv: 59
ECG duration for fp01_4.csv: 3734.52734375 seconds
Number of intervals for fp01_4.csv: 61
ECG duration for fp02_1.csv: 3721.453125 seconds
Number of intervals for fp02_1.csv: 61
ECG duration for fp02_2.csv: 3691.203125 seconds
Number of intervals for fp02_2.csv: 61
ECG duration for fp02_3.csv: 3575.078125 seconds
Number of intervals for fp02_3.csv: 59
ECG duration for fp02_4.csv: 3668.953125 seconds
Number of intervals for fp02_4.csv: 60
ECG duration for fp03_2.csv: 3990.5078125 seconds
Number of intervals for fp03_2.csv: 66
ECG duration for fp03_3.csv: 3580.953125 seconds
Number of intervals for fp03_3.csv: 59
ECG duration for fp03_4.csv: 3573.328125 seconds
Number of intervals for fp03_4.csv: 59
ECG duration for fp04_1.csv: 

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Loca

Error processing interval 69120-84480: cannot convert float NaN to integer
Error processing interval 84480-99840: cannot convert float NaN to integer
Error processing interval 99840-115200: cannot convert float NaN to integer
Error processing interval 115200-130560: cannot convert float NaN to integer
Error processing interval 130560-145920: integer division or modulo by zero
Error processing interval 145920-161280: index 0 is out of bounds for axis 0 with size 0
Error processing interval 161280-176640: integer division or modulo by zero
Error processing interval 176640-192000: integer division or modulo by zero
Error processing interval 192000-207360: integer division or modulo by zero
Error processing interval 207360-222720: integer division or modulo by zero
Error processing interval 222720-238080: integer division or modulo by zero
Error processing interval 238080-253440: integer division or modulo by zero
Error processing interval 253440-268800: integer division or modulo by zero


c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart

Error processing interval 852480-867840: integer division or modulo by zero
Error processing interval 867840-883200: integer division or modulo by zero
Error processing interval 883200-898560: integer division or modulo by zero
Error processing interval 898560-913920: integer division or modulo by zero
Error processing interval 913920-929280: integer division or modulo by zero
Error processing interval 929280-944640: integer division or modulo by zero
Error processing interval 944640-960000: integer division or modulo by zero
Error processing interval 960000-975360: integer division or modulo by zero
Number of intervals for fp13_1.csv: 2
ECG duration for fp13_2.csv: 3573.578125 seconds
Error processing interval 7680-23040: cannot convert float NaN to integer
Error processing interval 23040-38400: cannot convert float NaN to integer
Error processing interval 38400-53760: cannot convert float NaN to integer
Error processing interval 53760-69120: integer division or modulo by zero
Error p

...

Error processing interval 453120-468480: integer division or modulo by zero
Error processing interval 468480-483840: index 0 is out of bounds for axis 0 with size 0
Error processing interval 483840-499200: integer division or modulo by zero
Error processing interval 499200-514560: integer division or modulo by zero
Error processing interval 514560-529920: integer division or modulo by zero
Error processing interval 529920-545280: index 0 is out of bounds for axis 0 with size 0
Error processing interval 545280-560640: integer division or modulo by zero
Error processing interval 560640-576000: integer division or modulo by zero
Error processing interval 576000-591360: integer division or modulo by zero
Error processing interval 591360-606720: integer division or modulo by zero
Error processing interval 606720-622080: integer division or modulo by zero
Error processing interval 622080-637440: integer division or modulo by zero
Error processing interval 637440-652800: integer division or m

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Loca

Error processing interval 69120-84480: cannot convert float NaN to integer
Error processing interval 84480-99840: cannot convert float NaN to integer
Error processing interval 99840-115200: integer division or modulo by zero
Error processing interval 115200-130560: integer division or modulo by zero
Error processing interval 130560-145920: cannot convert float NaN to integer
Error processing interval 145920-161280: integer division or modulo by zero
Error processing interval 161280-176640: cannot convert float NaN to integer
Error processing interval 176640-192000: cannot convert float NaN to integer
Error processing interval 192000-207360: cannot convert float NaN to integer
Error processing interval 207360-222720: cannot convert float NaN to integer
Error processing interval 222720-238080: integer division or modulo by zero
Error processing interval 238080-253440: cannot convert float NaN to integer
Error processing interval 253440-268800: cannot convert float NaN to integer
Error pr

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Loca

Number of intervals for fp19_1.csv: 9
ECG duration for fp19_2.csv: 3582.71484375 seconds
Number of intervals for fp19_2.csv: 59
ECG duration for fp19_3.csv: 3015.7578125 seconds
Number of intervals for fp19_3.csv: 49
ECG duration for fp20_1.csv: 3712.078125 seconds
Number of intervals for fp20_1.csv: 61
ECG duration for fp20_2.csv: 3577.953125 seconds
Number of intervals for fp20_2.csv: 59
ECG duration for fp20_3.csv: 3045.82421875 seconds
Number of intervals for fp20_3.csv: 50
ECG duration for fp20_4.csv: 3628.453125 seconds
Number of intervals for fp20_4.csv: 59


## Save Features Dataset EDF

In [6]:
# concatenate all feature DataFrames into one DataFrame
all_features_df = pd.concat(feature_dfs, ignore_index=True)

# save the features DataFrame to a CSV file
all_features_df.to_csv('freq_domain_1_min_overlap.csv', index=False)

print("Saved features on 'freq_domain_1_min_overlap.csv'.")

Saved features on 'freq_domain_1_min_overlap.csv'.
